## MCP

In [ ]:
## Ensure Node & npx exist in WSL
!node -v
!npm -v
!npx -v

## If any of them show command not found, install Node:

# sudo apt update
# sudo apt install nodejs npm -y

v18.19.1
9.2.0
9.2.0


In [5]:
## run as administrator in terminal
# sudo npm install -g @modelcontextprotocol/server-filesystem
!pwd

/mnt/e/agentic-notebooks/4_mcp


In [3]:
!mkdir -p sandbox
!chmod -R 755 sandbox


In [11]:
!echo $PATH
!export PATH=/usr/bin:$PATH

/mnt/e/agentic-notebooks/.venv/bin:/home/tejaps/.cursor-server/bin/a9c77ceae65b77ff772d6adfe05f24d8ebcb2790/bin/remote-cli:/home/tejaps/.local/bin:/home/tejaps/.local/bin:/home/tejaps/.cursor-server/bin/a9c77ceae65b77ff772d6adfe05f24d8ebcb2790/bin/remote-cli:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/Program Files/Python313/Scripts/:/mnt/c/Program Files/Python313/:/mnt/c/Program Files/Eclipse Adoptium/jdk-21.0.8.9-hotspot/bin:/mnt/c/Program Files/Eclipse Adoptium/jre-21.0.8.9-hotspot/bin:/mnt/c/Windows/system32:/mnt/c/Windows:/mnt/c/Windows/System32/Wbem:/mnt/c/Windows/System32/WindowsPowerShell/v1.0/:/mnt/c/Windows/System32/OpenSSH/:/mnt/c/Program Files (x86)/NVIDIA Corporation/PhysX/Common:/mnt/c/Program Files/NVIDIA Corporation/NVIDIA NvDLISR:/mnt/c/Program Files/Git/cmd:/mnt/c/Program Files/Microsoft VS Code/bin:/mnt/c/Program Files/Docker/Docker/resources/bin:/mnt/c/Program Files/cursor/resources/app/bin:/mnt/c/

In [12]:
!which npx


/usr/bin/npx


In [17]:
!uname -a
!echo $SHELL
!pwd
!which python
!uname -a

Linux saiteja 6.6.87.2-microsoft-standard-WSL2 #1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux
/bin/bash
/mnt/e/agentic-notebooks/4_mcp
/mnt/e/agentic-notebooks/.venv/bin/python
Linux saiteja 6.6.87.2-microsoft-standard-WSL2 #1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux


In [ ]:
!alias npx="/usr/bin/npx"
!alias node="/usr/bin/node"
!alias npm="/usr/bin/npm"





Error accessing directory E:\agentic-notebooks\4_mcp\sandbox: [Error: ENOENT: no such file or directory, stat 'E:\agentic-notebooks\4_mcp\sandbox'] {
  errno: -2,
  code: 'ENOENT',
  syscall: 'stat',
  path: 'E:\\agentic-notebooks\\4_mcp\\sandbox'
}


In [22]:
# Make a clean sandbox in your Linux home dir
!mkdir -p ~/sandbox
!chmod -R 755 ~/sandbox

# Verify it exists
!ls -ld ~/sandbox

drwxr-xr-x 2 tejaps tejaps 4096 Oct 31 19:36 /home/tejaps/sandbox


In [23]:
!/usr/bin/npx @modelcontextprotocol/server-filesystem "/home/tejaps/sandbox"

Secure MCP Filesystem Server running on stdio
^C


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <h2 style="color:#ff7800;">Only runs in linux, to run in windows install WSL</h2>
            <span style="color:#ff7800;">
            </span>
        </td>
    </tr>
</table>

In [24]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner
from agents.mcp import MCPServerStdio
import os
from agents import set_tracing_export_api_key
from agents.tracing import trace 

In [26]:
load_dotenv(override=True)
os.environ["OPENAI_TRACE"] = "1"
set_tracing_export_api_key(os.getenv("OPENAI_API_KEY"))

### Steps for MCP in OpenAI Agents SDK

1. Create a Client

2. Have it spawn a server

3. Collect the tools that the server can use

Fetch mcp-server - fetches url from internetl and optionally extracts its contents as markdown

In [27]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

## Pre requisites, install node and playwrite

uv run playwright install --with-deps chromium

sudo apt install nodejs (in powershell WSL or linux)

sudo apt install npm (for npx in powershell WSL or linux)

## Playwrite MCP server - Javascript + node based MCP

In [28]:

playwright_params = {"command": "npx","args": [ "@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.list_tools()

playwright_tools


[Tool(name='browser_close', title=None, description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, icons=None, annotations=ToolAnnotations(title='Close browser', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_resize', title=None, description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, icons=None, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_console_mes

In [29]:

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "/home/tejaps/sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()

file_tools

[Tool(name='read_file', title=None, description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'type': 'number', 'description': 'If provided, returns only the last N lines of the file'}, 'head': {'type': 'number', 'description': 'If provided, returns only the first N lines of the file'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, icons=None, annotations=None, meta=None),
 Tool(name='read_text_file', title=None, description="Read the complete contents of a file from the file system as text. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a fil

### And now.. bring on the Agent with Tools!

In [30]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""


async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_server_browser:
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model="gpt-4.1-mini",
            mcp_servers=[mcp_server_files, mcp_server_browser]
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Find a great recipe for Banoffee Pie, then summarize it in markdown to banoffee.md")
            print(result.final_output)


I created a great Banoffee Pie recipe summary in markdown format for you. However, I am currently unable to save the file due to directory access restrictions.

Here is the markdown content for banoffee.md. You can copy and save it on your device:

```markdown
# Banoffee Pie Recipe

## Ingredients
- 250g digestive biscuits (crushed)
- 100g unsalted butter (melted)
- 2 x 397g cans of caramel (or dulce de leche)
- 3 ripe bananas (sliced)
- 300ml double cream
- 1 tbsp icing sugar
- Grated chocolate or cocoa powder (for garnish)

## Instructions

1. **Prepare the base:**
   - Mix crushed digestive biscuits with melted butter until combined.
   - Press the mixture into the base of a 23cm (9-inch) tart tin.
   - Chill in the fridge for at least 30 minutes to set.

2. **Add the caramel layer:**
   - Spread the caramel evenly over the biscuit base.

3. **Add the bananas:**
   - Arrange the sliced bananas over the caramel layer.

4. **Make the cream topping:**
   - Whip the double cream with ic

In [34]:
!ls "/home/tejaps/sandbox"

### Check out the trace

https://platform.openai.com/traces

### Now take a look at some MCP marketplaces

https://mcp.so

https://glama.ai/mcp

https://smithery.ai/

https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries

HuggingFace great community article:
https://huggingface.co/blog/Kseniase/mcp



